# TUTORIAL:
# Julia for technical computing

# Chapter 2: Types and type inference

We have already encounted a few types, including the numeric types `Int64`, `Float64`. **Writing Julia revolves a lot around types**. You will be making lots of them, and using them all the time. You can be generous with new types, they are not scarce. Sometimes code you write creates types implicitly just to get things done - in that case the type is essentially a use-once-throw-away commodity.

Just make sure that, at the end of the day, you still have $O(1)$ types and not $O(N)$: remember that Julia makes specialized versions of functions for each combination of types that you use for arguments. Adding types increases the size of your code and adds to the compilation overhead.

But what are *types*? All variables have a type. It is **compiler metadata** that describes what your variable stands for. Julia types compare a little bit to classes in an object-oriented language, but one should not stretch the comparison too much. Types do support inheritance. However, you are well advised to **resist that initial urge to create a fancy hierarchical tree of types like you would in OOP**: there is no need for that. The best reason to use types in Julia (and some would say the only reason) is to use *multiple dispatch*, and that is the topic of the next chapter.

Without multiple dispatch it is hard to convey the richness of the type system. Still, we have a quite a few things to discover already. As usual, the [manual](http://docs.julialang.org/en/release-0.4/manual/types/) is more complete than our set of examples.

### A note about development

You can not redefine existing types in the Julia workspace. This means that once you have defined `MyType`, you can not change its definition anymore. You can clear the current workspace with the command `workspace()`. While developing a larger program a good practice is to do all development inside a [module](http://docs.julialang.org/en/release-0.4/manual/modules/). You can reload a module and all definitions will be reloaded.

Another thing: for the time being, Julia does not keep track of code dependencies. If you change a function and pass it through the compiler again, Julia will not automatically recompile every other function which used the original verion of your function. It may have been inlined somewhere in its old version. This is different from Matlab, where if you change a function it will instantly be used everywhere. This behaviour could change in the future. If you write your code in a module, reloading the module is sufficient.

## 1. Type definition

### Composite types

You can compare a Julia type with a struct in C: it is an object that collects data. Member data are called fields. The data can be named and you define a type by listing the names of its fields.

You don't need to specify types of fields, but things will be more efficient when you do. Here is a first example:

In [1]:
type MyType
    a
end

The field `a` can be anything. Once you assign it a value, Julia will deduce its type. However, if the Julia compiler sees a `MyType` somewhere during compilation, it has no way to know in advance what the type of `a` will be, and so it will emit general (slower) code.

Here are some examples:

In [2]:
v = MyType(5)    # We instantiate an object of type MyType. There is a default constructor.

MyType(5)

In [3]:
v.a

5

In [4]:
typeof(v)

MyType

In [5]:
typeof(v.a)

Int64

In [6]:
v2 = MyType(5.0)

MyType(5.0)

In [7]:
typeof(v2)

MyType

In [8]:
typeof(v2.a)

Float64

In [9]:
v2.a = "Now I am a string"      # This is allowed. The type of a is not restricted, and it can change.

"Now I am a string"

In [10]:
v2

MyType("Now I am a string")

### Typed fields lead to more efficient code

Use untyped fields when you care about flexibility and simplicity more than performance. They do have unavoidable runtime overhead, so don't use untyped fields in a time-critical path of your code. In that case, do the following:

In [11]:
type MyType2
    a :: Float64
end

The type of field `a` is `Float64` and the compiler knows it: whenever an object of type `MyType2` comes along, Julia knows that field `a` is a `Float64` and so it will produce code accordingly.

In [12]:
v = MyType2(2)      # Note the integer I've supplied is automatically converted to a float

MyType2(2.0)

In [13]:
v.a = 4.0

4.0

In [14]:
v.a = "This won't work because I'm a string"

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Float64}, ::ASCIIString)
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert(::Type{Float64}, !Matched::Int8)
  convert(::Type{Float64}, !Matched::Int16)
  ...
while loading In[14], in expression starting on line 1

The last statement gave an error and it was obvious we were doing something that was not allowed. In general, Julia does not try very hard to prevent you from making mistakes. Julia tries to make your code run fast. The main reason for typing fields is not to prevent a user of your type from assigning values to it you were not expecting. The main reason is that the compiler knows *at compile-time* what the type of v.a is going to be, so that all uses of it can be optimized.

Of course, helpful error messages are, well, helpful. They can be expected to improve in time, as long as they don't impact speed. If you are expecting variables to have a certain type in your code and you want to enforce this, you can use [assertions](http://julia.readthedocs.org/en/latest/manual/types/#type-declarations).

Before we move on, I said typed fields are more efficient and expected you to believe it. But let's do the experiment.

In [15]:
function add_field_values_many_times(m)
    z = 0.0
    for i = 1:10000
        z = z + m.a
    end
end

add_field_values_many_times (generic function with 1 method)

We compare `MyType` and `MyType2`:

In [18]:
add_field_values_many_times(MyType(10.0))
@time add_field_values_many_times(MyType(10.0))

  0.000170 seconds (10.01 k allocations: 156.438 KB)


In [19]:
add_field_values_many_times(MyType2(10.0))
@time add_field_values_many_times(MyType2(10.0))

  0.000001 seconds (5 allocations: 176 bytes)


The timing difference is perhaps not enormous, but certainly more than significant. `MyType2` is faster than `MyType`, because in the former case Julia knows the type of the field `a`. Compare also the memory usage: manipulations with untyped fields lead to memory allocations, manipulations with typed fields apparently don't. At least in this case.

Julia tries to be fast even if you don't type your field values, and perhaps it will be faster still in the future. However, **if you help the compiler, the compiler helps you**. Make sure the compiler knows types of your variables, and more specialized and more efficient code will be the result.

### Immutable types

In [20]:
immutable Euro
    val  :: Float64
end

The `Euro` type is declared to be *immutable*. That means you can not change it after it has been instantiated. Knowing this allows for more compiler optimizations and simpler memory management. Any object of type `Euro` is, for all intents and purposes, equivalent to a `Float64`. The compiler will treat it as such, and a lot of the overhead of instantiating and manipulating types will be optimized away in the final machine instructions.

In [21]:
e = Euro(10)

Euro(10.0)

In [22]:
e.val = 12        # No can do - the object is immutable

LoadError: LoadError: type Euro is immutable
while loading In[22], in expression starting on line 1

In [23]:
e.val

10.0

In [24]:
typeof(e.val)

Float64

In [25]:
typeof(e)

Euro

How much space would an array of `Euro`'s take in memory? Let's check it out. Here is how much space an array of floating point numbers takes:

In [26]:
float_list = Array(Float64, 20)
sizeof(float_list)

160

And the same for `Euro`'s:

In [27]:
euro_list = Array(Euro, 20)
sizeof(euro_list)

160

It is exactly the same size in memory. There is no overhead to the `Euro` type in arrays: it acts like a list of floating point numbers.

We return to the `Euro` example shortly, in the next section.

### Singleton types

Types can also be completely empty. They are called *singleton types*. There is only one instance of it, which explains why in the code below `e == f` returns true.

In [28]:
immutable IAmEmpty
end

In [29]:
e = IAmEmpty()

IAmEmpty()

In [30]:
f = IAmEmpty()

IAmEmpty()

In [31]:
e==f

true

These types may look strange to you, since they contain no data. What is the point? They may remind you of static classes and static variables in C++, of which there is also always just one. And indeed we can use singleton types to achieve similar things. But more importantly, even without data, singleton **types carry some sort of meaning**, as Julia types frequently do.

Let's discuss what is the *meaning* of a type using the `Euro` example above. We'll do that in combination with functions.

## 2. Functions again, this time with types

### Types add meaning to variables

If the `Euro` type is pretty much equivalent to a `Float64`, why bother making a new type for it? Can't we make an alias for `Float64` and call that `Euro`, wouldn't that be simpler?

In this case, you can think of the type as adding *meaning* to the floating point value. **The type carries metadata**. This meaning exists only in the compiler, and for the most part only at compile-time. It is near invisible, and in particular cost-free, at runtime. But it allows you to express that certain functions, which manipulate floating point values, are meant to work with currency values.


You can make functions apply only to arguments of certain type, by adding the type after a double colon `::`. For example:

In [32]:
value(e::Euro) = e.val

value (generic function with 1 method)

In [33]:
e = Euro(10)
value(e)

10.0

If you write code passing around and manipulating `Euro`'s in terms of the `value` function, then your code is independent of the way this type stores the actual value. Good old computer science techniques there, **abstraction and encapsulation**! In most cases, the calls to `value` will be optimized away (inlined) after compilation, so there is no overhead. Abstraction without overhead could be a mantra of technical computing.

Note that in Julia you don't say `currency_value(e)`, you just say `value(e)`. The fact that it is about currency is encoded in the type of `e`, not in the function name. The word `value` is generic. It may apply to different things, not just to `Euro`.

You also don't say `e.value()`. In the Julia type system, the function `value` does not belong to `e` as in `e.value()`, it exists separately. Types have fields, which contain data, but **types do not have member functions**.

We can define what the `value` function does for arguments of different type. In that case, we may mean something different. **The combination of the function name and the types of its arguments determines the operation you want to perform.**

In [34]:
immutable Car
    price :: Float64
end

value(car::Car) = car.price

value (generic function with 2 methods)

In [35]:
c = Car(500)
value(c)

500.0

Julia knows you were asking about the value of a car, not of a Euro, because the type of `c` is `Car`. Julia selected the right function, the one we defined for `Car` and not the one we defined for `Euro`. This is what we mean by **dispatch**. For the time being, it looks like function overloading as in C or C++.

The following is just for fun. You can redefine the `show` function to modify how Julia displays your type. This function exists in a standard [module](http://docs.julialang.org/en/release-0.4/manual/modules/) called `Base`. I have used [string interpolation](http://docs.julialang.org/en/release-0.4/manual/strings#interpolation) in the process. I obtained the Euro symbol by typing \euro followed by `<TAB>` in the Jupyter notebook.

In [36]:
import Base: show

show(io::Base.IO, v::Euro) = print(io, "$(round(100*value(v))/100) €")

show (generic function with 111 methods)

In [37]:
Euro(3)

3.0 €

In [38]:
Euro(pi)

3.14 €

### Your own types are as fast as built-in types

There are other niceties. Your self-defined type is not inferior to any of the built-in types. For example, you can define addition, and it will be fast:

In [39]:
import Base: +

+(a::Euro, b::Euro) = Euro(value(a)+value(b))

+ (generic function with 156 methods)

In [40]:
a = Euro(2)
b = Euro(3)
a+b

5.0 €

Julia certainly wins some extra credits here: since we've defined `+` for our type, we can automatically use the `sum` function:

In [41]:
sum([a b])

5.0 €

That seems to work, so now let's measure speed.

In [47]:
N = 10000
float_list = rand(N)

s1 = sum(float_list)   # call sum(a) for warm-up, so compilation time is not part of our timing
@time sum(float_list)

euro_list = [Euro(float_list[i]) for i=1:length(float_list)]
s2 = sum(euro_list)
@time sum(euro_list)

4987.35 €

0.000005 seconds (5 allocations: 176 bytes)
  0.000007 seconds (5 allocations: 176 bytes)


See? I told you. Equally fast! Or, well, very very close, because I still see a difference, but it is tiny.

Summing `Euro`'s also does not allocate memory, at least not $O(N)$ memory. The small memory allocation shown above (176 bytes in my case) is actually due to how IPython executes things, not to the computation. So, summing `Euro`'s is **every bit as fast as if we were just adding floating point numbers**! 

We can also do the sum ourselves with a for loop, and still achieve comparable speed and avoid extensive memory allocations. Here is one way to do so. This is perhaps not how you would write it, and we will modify it later on.

In [48]:
function my_sum(a)
    z = a[1]
    for i = 2:length(a)
        z = z + a[i]
    end
    z
end

my_sum (generic function with 1 method)

In [49]:
s3 = my_sum(euro_list)
@time sum(float_list)
@time my_sum(euro_list)

4987.35 €

0.000010 seconds (5 allocations: 176 bytes)
  0.000014 seconds (5 allocations: 176 bytes)


Seems to be fine: our own sum is nearly as fast for `Euro`'s as the `sum` that comes with Julia is for floating point numbers.

### Built-in types are simply defined in Julia

In fact, even the core numeric types of Julia are (largely) defined in Julia. You can see them in the file [base/boot.jl](https://github.com/JuliaLang/julia/blob/master/base/boot.jl#L196), which contains the first code that is being executed whenever Julia is started. The arithmetic operations on floating points numbers are captured in [base/float.jl](https://github.com/JuliaLang/julia/blob/master/base/float.jl#L199).

This forces the compiler to make every type as fast as it can be, because the **built-in types are just implemented in Julia too**.

You can interact with them easily, for example to define multiplication.

In [50]:
import Base: *

*(a::Number, b::Euro) = Euro(a*value(b))

* (generic function with 139 methods)

In [51]:
2*Euro(2)

4.0 €

Alternatively, it is possible to hook up our `Euro` type into the promotion/conversion system that Julia has. Then the above definition would not have been necessary, things would just work. I won't do that here, but feel free to [explore](http://docs.julialang.org/en/release-0.4/manual/conversion-and-promotion/?highlight=conversion) and try it out yourself. You can make the `Euro` type blend in seamlessly in computations, and operations will be fast, while your variables carry meaning that differentiates them from regular floating point numbers.

## 3. Coming back to type stability

### Variables should retain their type

Not enough yet on this topic, let's return to that concept of type stability. The following code might have been your first shot at writing a sum routine, had I asked you (it was *my* first shot). It makes sense to put `z` to zero first, and then iterate over all elements of `a` to add them, rather than to iterate over all but the first element in the code above. Like this:

In [52]:
function my_sum_slow(a)
    z = 0
    for i = 1:length(a)
        z = z + a[i]
    end
    z
end

my_sum_slow (generic function with 1 method)

In [53]:
s4 = my_sum_slow(float_list)
@time sum(float_list)
@time my_sum_slow(float_list)

4987.354445007474

0.000008 seconds (5 allocations: 176 bytes)
  0.000269 seconds (20.00 k allocations: 312.656 KB)


Headaches! Headaches! It is much slower and memory usage is going through the roof! What happened here?

The reason is that `0` is an integer, but `a[1]` is not. The line `z = 0` makes the compiler infer (correctly) that `z` henceforth is an integer. But then as soon as we start adding `Float64`'s, `z` changes type to be a `Float64` itself. The code works as intendend, but this is a big no-no for performance. For best performance, **variables should retain their type throughout a function body**.

We can see exactly what Julia thinks it knows about the types of our variables.

In [54]:
@code_typed my_sum_slow(float_list)

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:a], Any[Any[Any[:a,Array{Float64,1},0],Any[:z,Any,2],Any[symbol("#s41"),Int64,2],Any[:i,Int64,18]],Any[],Any[UnitRange{Int64},Tuple{Int64,Int64},Int64,Int64,Int64],Any[]], :(begin  # In[52], line 2:
        z = 0 # In[52], line 3:
        GenSym(2) = (Base.arraylen)(a::Array{Float64,1})::Int64
        GenSym(0) = $(Expr(:new, UnitRange{Int64}, 1, :(((top(getfield))(Base.Intrinsics,:select_value)::I)((Base.sle_int)(1,GenSym(2))::Bool,GenSym(2),(Base.box)(Int64,(Base.sub_int)(1,1))::Int64)::Int64)))
        #s41 = (top(getfield))(GenSym(0),:start)::Int64
        unless (Base.box)(Base.Bool,(Base.not_int)(#s41::Int64 === (Base.box)(Base.Int,(Base.add_int)((top(getfield))(GenSym(0),:stop)::Int64,1))::Int64::Bool))::Bool goto 1
        2: 
        GenSym(3) = #s41::Int64
        GenSym(4) = (Base.box)(Base.Int,(Base.add_int)(#s41::Int64,1))::Int64
        i = GenSym(3)
        #s41 = GenSym(4) # In[52], line 4:
        z = z::Union{Float64,Int

This is a lot of gibberish, but note the final line: `end::Union{Float64,Int64}`. Julia can't make up its mind whether the final result is a Float64 or an Int64. That is not ideal, but things are even worse. Since `z` can also be both, during execution Julia allocates memory for a variable of type `Any`. That is, the variable `z` acts like a pointer to some memory location, and subsequent operations require a lot of allocations for new values of `z`.

You might think that the return value will always be Float64 if `float_list` is an array of Float64's. But this is not true: the length of the array could be zero, in which case the for loop is not executed and the result of the function is truly the integer `0`. So, the compiler is right. The code works and does what it should do, but it is slower. Not devastatingly slow, it just reduces to being Python-slow. Or Matlab-slow.

Expect future Julia IDE's to give you big fat warnings about this kind of type instability. It is easy to spot. You can already tell whether the compiler has succeeded in inferring all types and whether or not your code is type stable. It is a matter of functionality to automate this in a proper IDE. It is easy to write something that is not type stable. But, **if your function is not type-stable, Julia already knows.**

But wait, what if I use the list containing `Euro` values? See what happens.

In [55]:
@time my_sum_slow(euro_list)

LoadError: LoadError: MethodError: `+` has no method matching +(::Int64, ::Euro)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
  +(::Int64, !Matched::Int64)
  +(::Integer, !Matched::Ptr{T})
  ...
while loading In[55], in expression starting on line 155

We get an error. It is an error, because adding 0 (of type `Int64`) to a `Euro` is not possible. We did not define it. Or, as Julia says, "There is no method matching `+(::Int64, ::Euro)`".

In general, Julia does not favour automatic conversions. There is an elaborate [conversion system](http://docs.julialang.org/en/release-0.4/manual/conversion-and-promotion/?highlight=conversion), but this is opt-in. Automatic conversions often hide what is really going on, and that is not very Julian. We want to know *exactly* what our code does, and it should be either crystal clear or simple to find out.

### Problems with `my_sum`: what should be the type of the output?

One way to fix the type instability in `my_sum_slow` is to declare the type of `z`, like this:

In [56]:
function my_sum_not_slow_anymore(a)
    z::Float64 = 0
    for i = 1:length(a)
        z = z + a[i]
    end
    z
end

my_sum_not_slow_anymore (generic function with 1 method)

But this is ugly! After doing away with type declarations of variables, are we going to introduce them again through the backdoor for performance?

This implementation is also very restrictive. Why should the type of `z` be a `Float64` anyway? The former version did not assume anything of `a`, only that it (a) supports indexing and (b) supports addition of its elements. The argument `a` could contain integers, or `BigFloat`s, or `Euro`'s...

It is tempting to check the type of the first element of `a`, using `typeof(a[1])`. But then we're assuming that the array is non-empty. Empty arrays are a corner case for summation and corner cases matter in technical computing.

It is perfectly possible to create empty arrays of a certain type in Julia, so that it is always clear what the type of an empty array is. We create an empty array of `BigFloat`'s like this:

In [57]:
empty_array = Array(BigFloat,0)

0-element Array{BigFloat,1}

Julia's sum does the right thing, it returns a `BigFloat` zero:

In [58]:
sum(empty_array)

0.000000000000000000000000000000000000000000000000000000000000000000000000000000

In [59]:
typeof(ans)

BigFloat

But all of our own implementations of sum fail, either with an error or by responding with the wrong type:

In [60]:
my_sum(empty_array)

LoadError: LoadError: BoundsError: attempt to access 0-element Array{BigFloat,1}
  at index [1]
while loading In[60], in expression starting on line 1

In [61]:
my_sum_slow(empty_array)

0

In [62]:
typeof(ans)

Int64

In [63]:
my_sum_not_slow_anymore(empty_array)

0.0

In [64]:
typeof(ans)

Float64

In your Julia code, how do you find out about the type of an empty array? Or how do you find out the type of your parameters in general? There is a very elegant solution in Julia: parametric types and methods.

## 4. Parametric methods and parametric types

### Parametric types

We have already seen parametric types several times without naming them. Here is one of them:

In [65]:
a = Array(Float64, 4)

4-element Array{Float64,1}:
 6.89948e-310
 6.89948e-310
 6.89948e-310
 0.0         

In [66]:
typeof(a)

Array{Float64,1}

The type of `a` is Array, and it has two parameters: the type of the elements (Float64) and the dimension of the array (in this case 1).

You can create your own parametric types by adding type parameters to the definition:

In [67]:
immutable Point{T}
    x :: T
    y :: T
    z :: T
end

In [68]:
p = Point(0.1, 0.2, 0.4)

Point{Float64}(0.1,0.2,0.4)

Note that the description of the type of `p` includes the type parameter `Float64`:

In [69]:
typeof(p)

Point{Float64}

In [70]:
p.x

0.1

Why would I define a point this way? I want to specify the types of x, y and z, so that Julia knows them and produces optimized code. But I don't want to specify that they are `Float64`. What if my user wants to use `BigFloat`'s? Or integers? Or something else entirely I do not know about, some user-defined numeric type? In this case, parameters are the answer.

Type parameters in Julia look a lot like C++ template parameters. A major difference is that C++ templates are mostly syntactic sugar at compile-time. You could achieve what they do, if you have the patience, by using copy-paste of text over and over again. In Julia, parametric types and methods are a major part of the language in all stages of execution.

I am skipping over many things here, like default and user-supplied constructors. There are inner and outer constructors. They can be painful at times. Please read the manual on [constructors](http://docs.julialang.org/en/release-0.4/manual/constructors/), especially [parametric constructors](http://docs.julialang.org/en/release-0.4/manual/constructors/#parametric-constructors), and don't complain to me.

### Parametric methods

We have seen that you can specify the type of an argument in a function definition. Here is one way to fix our implementation of sum:

In [76]:
function my_sum2{T}(a::Array{T})
    z = zero(T)
    for i = 1:length(a)
        z += a[i]
    end
    z
end

my_sum2 (generic function with 1 method)

This is called a *parametric method*. The function `my_sum2` applies to all arrays that contain elements of type T. The dimension of the array can be anything, since I did not specify the second parameter of `Array`.

The parameter `T` is available in the function body. In particular, the line `z = zero(T)` creates a value that is numerically 0 but has type `T`. It is a convention to define this function for all types that can have numerical values.

In [77]:
my_sum2([2 3])

5

In [78]:
my_sum2([2.0 3.0])

5.0

In [79]:
my_sum2(Array(BigFloat, 0))

0.000000000000000000000000000000000000000000000000000000000000000000000000000000

Let's make our `Euro` type a good citizen in the Julia ecosystem and define `zero` for it. Since `zero` is defined in the module `Base`, we have to scope it correctly (remember to read more about [Modules](http://docs.julialang.org/en/release-0.4/manual/modules/)).

Note that `zero(T)` is a strange beast: the argument `T` is a type! Turns out that **you can pass around types as arguments**. Like functions and operators. Sure you can. In a modern language, how could it be any other way?

Here is how to accept a Type as an argument:

In [82]:
import Base: zero

zero(::Type{Euro}) = Euro(0.0)

zero (generic function with 14 methods)

In [83]:
my_sum2(Array(Euro, 0))

0.0 €

In [84]:
my_sum2([Euro(2) Euro(3)])

5.0 €

Types also have a type:

In [85]:
typeof(Float64)

DataType

In [86]:
typeof(Euro)

DataType

And DataType is its own type. Julia's type system is very consistent. People did think it through.

In [87]:
typeof(DataType)

DataType

## 5. Inheritance: abstract and concrete types

Sure enough, types can inherit from other types. You can create an abstract type and then inherit from it.

### Inheriting from abstract types

In [88]:
abstract AbstractPoint

Note that abstract types are always empty, they can not have fields. Also, you can never instantiate them. Each variable has a *concrete* type at any time.

Inheritance is specified using the `<:` symbol:

In [89]:
immutable Point2d <: AbstractPoint
    x :: Float64
    y :: Float64
end

In [90]:
immutable Point3d <: AbstractPoint
    x :: Float64
    y :: Float64
    z :: Float64
end

In [91]:
p2 = Point2d(0.1, 2.0)

Point2d(0.1,2.0)

In [92]:
typeof(p2)

Point2d

In [93]:
super(Point2d)

AbstractPoint

In [94]:
p3 = Point3d(5.0, 7.2, 4)

Point3d(5.0,7.2,4.0)

In [95]:
super(Point3d)

AbstractPoint

Any type that is not abstract is a *concrete type*. **You can not inherit from concrete types**. For example, we could not create a subset of `Point2d`'s, say a subset that has unit norm, as follows:

In [96]:
type Point2d_with_unit_norm <: Point2d
end

LoadError: LoadError: invalid subtyping in definition of Point2d_with_unit_norm
while loading In[96], in expression starting on line 1

Lots of issues arise when inheriting from concrete types. They make the code more complicated, and the compiler more complicated. Since you can achieve similar effects of what we were trying to do above in other ways in Julia, it is not as big a problem as it might seem at first.

A more important argument is the following, and it is about performance. Say you create an array of `Float64`s. It will be very memory-efficient, as we've already seen: the array just allocates enough memory to store its elements. You don't want a user making a type that inherits from `Float64` and then try to put that type in your array. It would not be possible without making all elements of the array pointers to elsewhere by default, in order to resolve the ambiguity or the possible mismatch in size. This adds indirection, which makes it slow.

### When to use (or not to use) inheritance

Another good old computer science technique is **code reuse**. In other words, don't duplicate code.

This implies defining a function on the most general level where it can be defined. For example, since both `Point2d` and `Point3d` have a field called `x`, we might want to have a shared implementation of a function that returns the value of `x`:

In [97]:
first_dimension(p::AbstractPoint) = p.x

first_dimension (generic function with 1 method)

In [100]:
p2 = Point2d(0.1, 2.0)
first_dimension(p2)

0.1

In [101]:
p3 = Point3d(5.0, 7.2, 4)
first_dimension(p3)

5.0

A lot of things are happening here:
* There is not just one version of the `first_dimension` function. Recall that Julia compiles a version of your function that is specific to the types of the arguments that you give it. So, different code results for `p2` and `p3`. The source code, however, is the same for both cases and that is why we have expressed it at the level of `AbstractPoint`.
* We wrote `p.x`, even though `AbstractPoint` has no field `x`. The code does not seem to make sense. But who cares? You can not supply an argument with an abstract type to the function anyway. As long as the concrete type you use has a field called `x`, Julia will accept it.
* How does Julia know which version of the function to call in our examples with `p2` and `p3`? It looks at the type of the argument. This is *type-based dispatch*. In a larger program, if the compiler was able to infer the type of the argument variable without any ambiguity, then there is no need to check at runtime and your code immediately jumps to the right set of instructions. If not, Julia performs a type-check at runtime, and *then* calls the right function. This means the same code is executed, whether the compiler knew about the type of your variable ahead of time or not: this is the meaning of *dynamic* in **dynamic dispatch**.

Now, back to inheritance. Is code reuse a good enough reason to use it?

Consider the following:

In [103]:
second_dimension(p) = p.y

second_dimension (generic function with 1 method)

In [104]:
second_dimension(p2)

2.0

In [105]:
second_dimension(p3)

7.2

Hmmm. This does exactly the same as the `first_dimension` function, but I did not specify the `AbstractPoint` type! In fact, I specified no type at all. This is commonly called **duck-typing**.

In Python and Matlab, duck typing is the only possibility. In Julia, you can add types, but you don't have to. However, duck typing is strongly encouraged: it emposes the fewest restrictions on later use of your code. Julia will happily apply `second_dimension` to any object of some type that has `y` as a field name, including `Point2d` and `Point3d`, but perhaps also others. Even though Julia uses types extensively, it does not try to force them upon you needlessly. As long as things work then, well, they work. Everybody happy.

But there are cases when you can not rely on duck typing. My function name `second_dimension` is rather generic. Imagine if I write the following code later on:

In [106]:
type EinsteinianSpacetime
    space1
    space2
    space3
    time
end

second_dimension(e::EinsteinianSpacetime) = e.space2

second_dimension (generic function with 2 methods)

I could not use duck typing a second time, because I want `e.space2` to be returned when `e` is an `EinsteinianSpaceTime`, and not `e.y`. Duck typing has its limits. Essentially, you can only use it once. Having a function with two different meanings, depending on the types of the variables, is a good reason to type your arguments. In fact, this is an important lesson, so let's use bold to make sure you read this. **Using dispatch is the only reason for adding types to your arguments.** It. Is. The. Only. Reason. Not performance. Not documentation. Dispatch.

Did we need inheritance to achieve code reuse? Strictly speaking, no, we did not.

### Things Julia doesn't let you do

It is clear that the kind of inheritance that Julia supports is much more limited than in most OOP languages.
1. Abstract types can not have fields.
2. You can not inherit from concrete types.
3. There is (currently) no support for multiple inheritance (and there might never be).
4. There is (currently) no support for interfaces.

It turns out that the world of Julia does not come to a complete stop because of this and, in fact, life is not unpleasant without these features. There are alternatives. A topic too advanced for this tutorial are so-called **traits**. I am not supplying a link this time, because you have to read on first.

## 6. Numeric types in Julia

Since you are into technical computing, you probably want to know about the numeric types that Julia supports natively. See the manual on [Integers and floating point numbers](http://docs.julialang.org/en/release-0.4/manual/integers-and-floating-point-numbers/) and [Complex and rational numbers](http://docs.julialang.org/en/latest/manual/complex-and-rational-numbers/). See also the exact definitions in [base/boot.jl](https://github.com/JuliaLang/julia/blob/master/base/boot.jl#L182).

### Sizes

In [109]:
sizeof(Float64)

8

In [110]:
sizeof(Float32)

4

In [111]:
sizeof(Float16)

2

In [112]:
sizeof(Int64)

8

In [113]:
sizeof(Int32)

4

In [114]:
sizeof(Int16)

2

In [115]:
sizeof(UInt64)

8

### Type hierarchy

In [116]:
super(Float64)

AbstractFloat

In [117]:
super(AbstractFloat)

Real

In [118]:
super(Real)

Number

In [119]:
super(Number)

Any

In [120]:
super(Int64)

Signed

In [121]:
super(Signed)

Integer

In [122]:
super(Integer)

Real

In [123]:
super(Real)

Number

In [124]:
super(UInt64)

Unsigned

In [125]:
super(Unsigned)

Integer

### There is no runtime cost to arguments with abstract types

You can make a function that applies only to numbers as follows:

In [126]:
square(x::Number) = x*x

square (generic function with 1 method)

**There is no performance penalty for using abstract types**. Recall that at each function call Julia calls a compiled version of that function that is specific for the types of the arguments. The types of arguments are always concrete, since you can not instantiate abstract types. If Julia knows the concrete type of your variable at compile-time, it will produce specialized and efficient code for `square(x)`.